In [0]:
import os
import itertools
import matplotlib
import matplotlib.pyplot as plt
  
from random import randint, seed
import itertools
import numpy as np
import cv2

%matplotlib inline

In [0]:
# Mask generator obtained from 
# https://github.com/MathiasGruber/PConv-Keras/blob/master/libs/util.py

class MaskGenerator():

    def __init__(self, height, width, channels=3, rand_seed=None, filepath=None):
        """Convenience functions for generating masks to be used for inpainting training
        
        Arguments:
            height {int} -- Mask height
            width {width} -- Mask width
        
        Keyword Arguments:
            channels {int} -- Channels to output (default: {3})
            rand_seed {[type]} -- Random seed (default: {None})
            filepath {[type]} -- Load masks from filepath. If None, generate masks with OpenCV (default: {None})
        """

        self.height = height
        self.width = width
        self.channels = channels
        self.filepath = filepath

        # If filepath supplied, load the list of masks within the directory
        self.mask_files = []
        if self.filepath:
            filenames = [f for f in os.listdir(self.filepath)]
            self.mask_files = [f for f in filenames if any(filetype in f.lower() for filetype in ['.jpeg', '.png', '.jpg'])]
            print(">> Found {} masks in {}".format(len(self.mask_files), self.filepath))        

        # Seed for reproducibility
        if rand_seed:
            seed(rand_seed)

    def _generate_mask(self):
        """Generates a random irregular mask with lines, circles and elipses"""

        img = np.zeros((self.height, self.width, self.channels), np.uint8)

        # Set size scale
        size = int((self.width + self.height) * 0.03)
        if self.width < 64 or self.height < 64:
            raise Exception("Width and Height of mask must be at least 64!")
        
        # Draw random lines
        for _ in range(randint(1, 20)):
            x1, x2 = randint(1, self.width), randint(1, self.width)
            y1, y2 = randint(1, self.height), randint(1, self.height)
            thickness = randint(3, size)
            cv2.line(img,(x1,y1),(x2,y2),(1,1,1),thickness)
            
        # Draw random circles
        for _ in range(randint(1, 20)):
            x1, y1 = randint(1, self.width), randint(1, self.height)
            radius = randint(3, size)
            #cv2.circle(img,(x1,y1),radius,(1,1,1), -1)
            
        # Draw random ellipses
        for _ in range(randint(1, 20)):
            x1, y1 = randint(1, self.width), randint(1, self.height)
            s1, s2 = randint(1, self.width), randint(1, self.height)
            a1, a2, a3 = randint(3, 180), randint(3, 180), randint(3, 180)
            thickness = randint(3, size)
            #cv2.ellipse(img, (x1,y1), (s1,s2), a1, a2, a3,(1,1,1), thickness)
        
        return 1-img

    def _load_mask(self, rotation=True, dilation=True, cropping=True):
        """Loads a mask from disk, and optionally augments it"""

        # Read image
        mask = cv2.imread(os.path.join(self.filepath, np.random.choice(self.mask_files, 1, replace=False)[0]))
        
        # Random rotation
        if rotation:
            rand = np.random.randint(-180, 180)
            M = cv2.getRotationMatrix2D((mask.shape[1]/2, mask.shape[0]/2), rand, 1.5)
            mask = cv2.warpAffine(mask, M, (mask.shape[1], mask.shape[0]))
            
        # Random dilation
        if dilation:
            rand = np.random.randint(5, 47)
            kernel = np.ones((rand, rand), np.uint8) 
            mask = cv2.erode(mask, kernel, iterations=1)
            
        # Random cropping
        if cropping:
            x = np.random.randint(0, mask.shape[1] - self.width)
            y = np.random.randint(0, mask.shape[0] - self.height)
            mask = mask[y:y+self.height, x:x+self.width]

        return (mask > 1).astype(np.uint8)

    def sample(self, random_seed=None):
        """Retrieve a random mask"""
        if random_seed:
            seed(random_seed)
        if self.filepath and len(self.mask_files) > 0:
            return self._load_mask()
        else:
            return self._generate_mask()

In [0]:
from google.colab import files

In [0]:
uploaded = files.upload()

Saving 1041x1162.png to 1041x1162.png


In [0]:
import io
from PIL import Image
im_list = []
imageFilename = ['men.png', 'bird.png', 'pc.png', '1158x1041.png', '1041x1162.png']

for im in imageFilename:
  imageFileObj = open(im, "rb")
  imageBinaryBytes = imageFileObj.read()
  imageStream = io.BytesIO(imageBinaryBytes)
  imageFile = Image.open(imageStream)
  im_list.append(imageFile)

In [0]:
# Telea and NS for the image with a man

arr_men = np.array(im_list[0])[:,:,0:3]
# Instantiate mask generator
mask_generator = MaskGenerator(256, 256, 1, rand_seed=21)

mask = mask_generator.sample()
mask = (1-mask.reshape(256,256))*255
img = Image.fromarray(mask,"L")
img.save('men_mask.png')
files.download('men_mask.png')

dst = cv2.inpaint(arr_men, mask,3,cv2.INPAINT_TELEA)
img = Image.fromarray(dst,"RGB")
img.save('men_Telea.png')
files.download('men_Telea.png')

dst = cv2.inpaint(arr_men, mask,3,cv2.INPAINT_NS)
img = Image.fromarray(dst,"RGB")
img.save('men_NS.png')
files.download('men_NS.png')

mask_generator = MaskGenerator(256, 256, 3, rand_seed=21)
mask = (1 - mask_generator.sample())*255
#masked_men = (arr_men < mask)*arr_men + (mask > arr_men)*255
masked_men = np.maximum(mask, arr_men)
img = Image.fromarray(masked_men,"RGB")
img.save('men_masked.png')
files.download('men_masked.png')

In [0]:
# Telea and NS for the image with a bird
arr = np.array(im_list[1])[:,:,0:3]
name = 'bird'
num = 18
# Instantiate mask generator

mask_generator = MaskGenerator(256, 256, 1, rand_seed=14)
mask = mask_generator.sample()
mask = (1-mask.reshape(256,256))*255
img = Image.fromarray(mask,"L")
img.save(name + '_mask.png')
files.download(name + '_mask.png')


dst = cv2.inpaint(arr, mask,3,cv2.INPAINT_TELEA)
img = Image.fromarray(dst,"RGB")
img.save(name + '_Telea.png')
files.download(name + '_Telea.png')

dst = cv2.inpaint(arr, mask,3,cv2.INPAINT_NS)
img = Image.fromarray(dst,"RGB")
img.save(name + '_NS.png')
files.download(name + '_NS.png')

mask_generator = MaskGenerator(256, 256, 3, rand_seed=14)
mask = (1 - mask_generator.sample())*255
masked_men = np.maximum(mask, arr)
img = Image.fromarray(masked_men,"RGB")
img.save(name + '_masked.png')
files.download(name + '_masked.png')

In [0]:
# Telea and NS for the image with a pc
arr = np.array(im_list[2])[:,:,0:3]
name = 'pc'
num = 18
# Instantiate mask generator

mask_generator = MaskGenerator(256, 256, 1, rand_seed=num)
mask = mask_generator.sample()
mask = (1-mask.reshape(256,256))*255
img = Image.fromarray(mask,"L")
img.save(name + '_mask.png')
files.download(name + '_mask.png')


dst = cv2.inpaint(arr, mask,3,cv2.INPAINT_TELEA)
img = Image.fromarray(dst,"RGB")
img.save(name + '_Telea.png')
files.download(name + '_Telea.png')

dst = cv2.inpaint(arr, mask,3,cv2.INPAINT_NS)
img = Image.fromarray(dst,"RGB")
img.save(name + '_NS.png')
files.download(name + '_NS.png')

mask_generator = MaskGenerator(256, 256, 3, rand_seed=num)
mask = (1 - mask_generator.sample())*255
masked_men = np.maximum(mask, arr)
img = Image.fromarray(masked_men,"RGB")
img.save(name + '_masked.png')
files.download(name + '_masked.png')

In [0]:
# Telea and NS for the image of Medellin
arr = np.array(im_list[3])[:,:,0:3]
name = 'medallo'
num = 5650000
# Instantiate mask generator

mask_generator = MaskGenerator( 1041, 1158, 1, rand_seed=num)
mask = mask_generator.sample()
mask = (1-mask.reshape(1041, 1158))*255
img = Image.fromarray(mask,"L")
img.save(name + '_mask.png')
files.download(name + '_mask.png')


dst = cv2.inpaint(arr, mask,3,cv2.INPAINT_TELEA)
img = Image.fromarray(dst,"RGB")
img.save(name + '_Telea.png')
files.download(name + '_Telea.png')

dst = cv2.inpaint(arr, mask,3,cv2.INPAINT_NS)
img = Image.fromarray(dst,"RGB")
img.save(name + '_NS.png')
files.download(name + '_NS.png')

mask_generator = MaskGenerator(1041, 1158, 3, rand_seed=num)
mask = (1 - mask_generator.sample())*255
masked_men = np.maximum(mask, arr)
img = Image.fromarray(masked_men,"RGB")
img.save(name + '_masked.png')
files.download(name + '_masked.png')

In [0]:
# Telea and NS for the image of Medellin grasslands
arr = np.array(im_list[4])[:,:,0:3]
name = 'medallo_grasslands'
num = 5650001
# Instantiate mask generator

mask_generator = MaskGenerator( 1041, 1162, 1, rand_seed=num)
mask = mask_generator.sample()
mask = (1-mask.reshape(1041, 1162))*255
img = Image.fromarray(mask,"L")
img.save(name + '_mask.png')
files.download(name + '_mask.png')


dst = cv2.inpaint(arr, mask,3,cv2.INPAINT_TELEA)
img = Image.fromarray(dst,"RGB")
img.save(name + '_Telea.png')
files.download(name + '_Telea.png')

dst = cv2.inpaint(arr, mask,3,cv2.INPAINT_NS)
img = Image.fromarray(dst,"RGB")
img.save(name + '_NS.png')
files.download(name + '_NS.png')

mask_generator = MaskGenerator(1041, 1162, 3, rand_seed=num)
mask = (1 - mask_generator.sample())*255
masked_men = np.maximum(mask, arr)
img = Image.fromarray(masked_men,"RGB")
img.save(name + '_masked.png')
files.download(name + '_masked.png')